In [1]:
import torch
from torch import nn, optim
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
import jieba.posseg as pseg
import re

In [ ]:
en_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
en_model = AutoModel.from_pretrained("bert-base-cased")
zh_tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
zh_model = AutoModel.from_pretrained("bert-base-chinese")

In [3]:
df1 = pd.read_csv('./notification_sorted.csv')

r1 = u'[a-zA-Z0-9’!"#$%&\'()*+「」,-./:;<=>?°·@，。?★、…【】《》？“”‘’［］！[\\]^_`{|}~]+'
r2 = "[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+"
r3 =  "[.!//_,$&%^*()<>+\"'?@#-|:~{}]+|[——！\\\\，。=？、：“”‘’《》【】￥……（）]+" 
r4 =  "\\【.*?】+|\\《.*?》+|\\#.*?#+|[.!/_,$&%^*()<>+""'?@|:~{}#]+|[——！\\\，。=？、：“”‘’￥……（）《》【】]"

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F9D0"
                           "]+", flags=re.UNICODE)
        
droplist = list()        
for i in range(len(df1)):

    if len(str(df1.iloc[i]['appName'])) == 0 or len(str(df1.iloc[i]['category'])) == 0 or len(str(df1.iloc[i]['title'])) == 0 or len(str(df1.iloc[i]['content'])) == 0:
        droplist.append(i)
        continue

    sentence = re.sub(r4,'',str(df1.iloc[i]['content']))
    sentence = re.sub(r1,'',sentence)
    sentence = re.sub(r" ","",sentence)
    sentence = re.sub("\n","",sentence)
    sentence = re.sub("\u200b","",sentence)
    sentence = re.sub("\u200d","",sentence)
    sentence = emoji_pattern.sub(r'', sentence)
    sentence = re.sub(r"http\S+", "", sentence)

    if len(sentence) == 0:
        droplist.append(i)
        continue

    words = pseg.cut(sentence)
    sentence = ' '.join([word for word, flag in words if flag != 'x'])

    if len(sentence) == 0:
        droplist.append(i)
        continue

    df1.loc[i,'content'] = sentence

df1.drop(droplist, axis=0, inplace=True)

text = df1[['appName', 'category', 'title', 'content']].values.tolist()
text

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.cache
Loading model cost 0.683 seconds.
Prefix dict has been built successfully.


[['LINE', 'IM', '2b8e91362efa639', '哦哦哦 原來 如此'],
 ['Messenger', 'IM', '2198a1d47e55759', '是'],
 ['地圖', 'Transportation', '完成你的時間軸', '新增 你 星期日 在 頭份 遺漏 的 造訪 記錄'],
 ['Google', 'Weather', 'Google 助理語言', '你 隨時 可 前往 設定 頁面 更新 助理 語言'],
 ['地圖',
  'Transportation',
  '造訪 🧳 Shinn Ru Dormitory的感覺如何？',
  '你 很 有人 氣 你 的 評論 獲得 超過 次觀 看'],
 ['Google', 'Weather', '明日預報 · 新竹', '雷陣雨 查看 完整 預報'],
 ['LINE', 'IM', 'LINE', '透過 電話 號碼 將 您 加為 好友'],
 ['PChome24h購物', 'Shopping', '夏日出遊趣!介紹女神私藏品牌', '瑪莉 菲絲 廖小安 尹樂 聯合 推薦'],
 ['相簿',
  'Tool',
  '保持井然有序',
  '封存 相片 檢視 畫面 中 內 含 文件 和 螢幕 截圖 的 相片 你 隨時 可以 透過 導覽 選單 存取 封存 檔案'],
 ['台新銀行',
  'Finance',
  '台新銀行通知',
  '帳務 通知 您 的 帳戶 末 四碼 於 有 金額 扣款 建議 您 至 帳戶 總覽 查詢 如有 疑問 請洽 客服'],
 ['Richart', 'Finance', 'Richart', '你 在 從 帳戶 轉出 元 對方 帳號 末 四碼'],
 ['LINE', 'IM', 'LINE貼圖最前線', '倒數 小 時 撒 嬌 鬼 小可 愛 在 這裡 等 你 把握 搶購 折貼 圖 的 最 後 機會'],
 ['下載管理員', 'System', 'app-debug.apk', '下載 完成'],
 ['Android 系統', 'System', '數據用量警告', '輕觸 即可 查看 使用量 和 設定'],
 ['WeChat', 'IM', '群組', '條 好 的'],
 ['Labeling Study', 'Tool', '

In [6]:
text = df1[['appName', 'category', 'title', 'content']].values.astype(str).tolist()
X = torch.stack([torch.cat((
        torch.cat(en_model(**en_tokenizer(row, return_tensors='pt', padding=True, truncation=True), output_hidden_states=True)[2][-4: ])[:, 0].detach(),
        torch.cat(zh_model(**zh_tokenizer(row, return_tensors='pt', padding=True, truncation=True), output_hidden_states=True)[2][-4: ])[:, 0].detach(),
            )) for row in tqdm(text)])
y = torch.LongTensor([6 if np.isnan(label) else int(label) for label in df1['display_order'].values.tolist()])

100%|██████████| 15483/15483 [1:57:03<00:00,  2.20it/s]  


In [7]:
print(X.shape)
print(y.shape)
torch.save(X, 'X.pt')
torch.save(y, 'y.pt')

torch.Size([15483, 32, 768])
torch.Size([15483])
